In [119]:
#import some stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from tqdm import tqdm
import glob
import time
import xgboost as xgb
from sklearn.model_selection import train_test_split #use this?
from scipy import stats

In [458]:
urlScout = 'https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_scouting.csv'
urlWomen = 'https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_womens.csv'
urlNWHL = 'https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_nwhl.csv'
dataScout = pd.read_csv(urlScout)
dataWomen = pd.read_csv(urlWomen)
dataNWHL = pd.read_csv(urlNWHL)

In [3]:
pd.set_option('display.max_columns', None)

In [462]:
len(dataScout)

75873

In [389]:
dataScout.head(1)

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2019-09-20,Erie Otters,Sudbury Wolves,1,20:00,5,5,0,0,Sudbury Wolves,Blake Murray,Faceoff Win,100,43,Forehand,NaN,NaN,NaN,Connor Lockhart,NaN,NaN


In [461]:
a = np.array(dataWomen[['game_date','Home Team','Away Team']]).astype("<U22")
print('There are ',len(np.unique(a,axis=0)),' games in the women dataset')
b = np.array(dataScout[['game_date','Home Team','Away Team']]).astype("<U22")
print('There are ',len(np.unique(b,axis=0)),' games in the scout dataset')
b = np.array(dataNWHL[['game_date','Home Team','Away Team']]).astype("<U22")
print('There are ',len(np.unique(b,axis=0)),' games in the NWHL dataset')

There are  13  games in the women dataset
There are  40  games in the scout dataset
There are  15  games in the NWHL dataset


In [261]:
dataWomen.head(1)

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2018-02-11,Olympic (Women) - Canada,Olympic (Women) - Olympic Athletes from Russia,1,20:00,5,5,0,0,Olympic (Women) - Canada,Emily Clark,Faceoff Win,100,42,Backhand,NaN,NaN,NaN,Lyudmila Belyakova,NaN,NaN


In [339]:
print('Amount of Passes for Women:',len(dataWomen.loc[(dataWomen['Event'] ==  'Play' ) | (dataWomen['Event'] == 'Incomplete Play')]))
print('Amount of Shots for Women:',len(dataWomen.loc[(dataWomen['Event'] ==  'Shot' ) | (dataWomen['Event'] == 'Goal')]))

Amount of Passes for Women: 10105
Amount of Shots for Women: 1671


In [340]:
print('Amount of Passes for Scout:',len(dataScout.loc[(dataScout['Event'] ==  'Play' ) | (dataScout['Event'] == 'Incomplete Play')]))
print('Amount of Shots for Scout:',len(dataScout.loc[(dataScout['Event'] ==  'Shot' ) | (dataScout['Event'] == 'Goal')]))

Amount of Passes for Scout: 32668
Amount of Shots for Scout: 5180


In [8]:
#dataWomen.loc[(dataWomen['Event'] == 'Zone Entry') & (dataWomen['Detail 1'] == 'Played')].head()

In [366]:
dataScout.loc[(dataScout['Event'] ==  'Shot' ) | (dataScout['Event'] == 'Goal')].head(1)

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
16,2019-09-20,Erie Otters,Sudbury Wolves,1,19:26,5,5,0,0,Sudbury Wolves,Shane Bulitka,Shot,177,57,Snapshot,On Net,f,f,NaN,NaN,NaN


In [420]:
#here this will help make an xGoals model
def getDataReadyxGoals(df):
    df = df.copy()
    df['isFaceOff'] = df['Event'].apply(lambda row: row == 'Faceoff Win')
    df['SecondsLeft'] = df['Clock'].apply(lambda row: float(row.split(':')[0]) * 60 + float(row.split(':')[1]))
    df['ChangeInPoss'] = (df['Team'] != df['Team'].shift(1))
    df['ChangeInPoss'] = df.apply(lambda row: True if (row['Event'] == 'Faceoff Win') else row['ChangeInPoss'],axis=1)
    df = df.rename(columns={'ChangeInPoss':'PossStart'})
    df['PossStartTime'] = df.apply(lambda row: row['SecondsLeft'] if row['PossStart'] else -1.0,axis=1)
    
    
    for i in range(len(df)):
        if df.loc[i,'PossStartTime'] == -1.0:
            df.loc[i,'PossStartTime'] = df.loc[i-1,'PossStartTime']
    df['PossTotalTime'] = df.apply(lambda row: row['PossStartTime'] - row['SecondsLeft'],axis=1)
    df['LastEvent'] = df['Event'].shift(1)
    df['TimeBetweenEvents'] = df['SecondsLeft'].shift(1,fill_value=1200) - df['SecondsLeft']
    df = df.drop(columns=['PossStart','PossStartTime']) #drop these cuz I don't think I need them
    
    #filteredDF = df.loc[(df['Event'] ==  'Shot' ) | (df['Event'] == 'Goal')].copy() #only need these rows
    filteredDF = df.copy()
    filteredDF['fromPass'] = filteredDF.apply(lambda row: 1 if (row['LastEvent'] == 'Play') else 0,axis=1)
    filteredDF['TeamSkaters'] = filteredDF.apply(lambda row: row['Home Team Skaters'] if (row['Team'] == row['Home Team']) else row['Away Team Skaters'],axis=1)
    filteredDF['OppSkaters'] = filteredDF.apply(lambda row: row['Home Team Skaters'] if (row['Team'] == row['Away Team']) else row['Away Team Skaters'],axis=1)
    filteredDF['onPP'] = filteredDF.apply(lambda row: row['TeamSkaters'] > row['OppSkaters'],axis=1)
    filteredDF['onPK'] = filteredDF.apply(lambda row: row['TeamSkaters'] < row['OppSkaters'],axis=1)
    filteredDF['AmountExtraSkaters'] = filteredDF.apply(lambda row: row['TeamSkaters'] - row['OppSkaters'],axis=1)
    filteredDF['Goal'] = filteredDF.apply(lambda row: 1 if (row['Event'] == 'Goal') else 0,axis=1)
    filteredDF['TypeDeflection'] = filteredDF.apply(lambda row: 1 if (row['Detail 1'] == 'Deflection') else 0,axis=1)
    filteredDF['TypeSlapshot'] = filteredDF.apply(lambda row: 1 if (row['Detail 1'] == 'Slapshot') else 0,axis=1)
    filteredDF['TypeSnapshot'] = filteredDF.apply(lambda row: 1 if (row['Detail 1'] == 'Snapshot') else 0,axis=1)
    filteredDF['TypeWrapAround'] = filteredDF.apply(lambda row: 1 if (row['Detail 1'] == 'Wrap Around') else 0,axis=1)
    filteredDF['TypeWristshot'] = filteredDF.apply(lambda row: 1 if (row['Detail 1'] == 'Wristshot') else 0,axis=1)
    #we don't include detail 2 since that would bias the results toward 'on net' being the best
    #it's info we don't know at the shot time
    filteredDF['isTraffic'] = filteredDF.apply(lambda row: 1 if (row['Detail 3'] == 't') else 0,axis=1)
    filteredDF['isOneTimer'] = filteredDF.apply(lambda row: 1 if (row['Detail 4'] == 't') else 0,axis=1)
    filteredDF['DistanceToCenterGoal'] = filteredDF.apply(lambda row: ((189 - row['X Coordinate'])**2 + (42.5 - row['Y Coordinate'])**2)**0.5,axis=1)
    def getAreaNet(x,y):
        if y > 42.5:
            y = 85 - y #so it's like we're on the bottom side of the rink - mirror doesn't matter
        hyp = ((189 - x)**2 + (39.5 - y)**2)**0.5
        vector1 = [189 - x, 39.5 - y]
        vector2 = [189 - x, 45.5 - y]
        unit_vector_1 = vector1 / np.linalg.norm(vector1)
        unit_vector_2 = vector2 / np.linalg.norm(vector2)
        dot_product = np.dot(unit_vector_1, unit_vector_2)
        angle = np.arccos(dot_product)
        angle = angle / 2 #using angle bisector for triangle
        halfNet = np.sin(angle) * hyp
        fullNet = 2 * halfNet
        return fullNet*4
    filteredDF['SurfaceAreaNetSeen'] = filteredDF.apply(lambda row: getAreaNet(row['X Coordinate'],row['Y Coordinate']),axis=1)
    
    filteredDF['GoalsDown'] = filteredDF.apply(lambda row: (row['Home Team Goals'] - row['Away Team Goals'])
                                              if (row['Team'] == row['Away Team Goals'])
                                              else (row['Away Team Goals'] - row['Home Team Goals']),axis=1)
    #filteredDF = filteredDF[['PossTotalTime','onPP','onPK','AmountExtraSkaters','fromPass','TimeBetweenEvents',
    #                  'TypeDeflection','TypeSlapshot','TypeSnapshot','TypeWrapAround','TypeWristshot',
    #                  'isTraffic','isOneTimer','DistanceToCenterGoal','SurfaceAreaNetSeen','Goal']] #not using GoalsDown
    
    #this is for the small df
    #filteredDF = filteredDF[['PossTotalTime','onPP','onPK','AmountExtraSkaters','DistanceToCenterGoal','SurfaceAreaNetSeen','Goal']]
    #X = filteredDF.iloc[:,:-1]
    #y = filteredDF.iloc[:,-1]
    return filteredDF #X,y

In [421]:
testDF = getDataReadyxGoals(dataScout)

In [135]:
#get train/test split ready

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.25,random_state=24) #can add shuffle=False to not shuffle

an idea is to train on non-erie player and test on erie

try XGBRegressor or classifier?

In [442]:
print('There were',len(testDF.loc[(testDF['Event'] == 'Penalty Taken') & (testDF['Detail 1'] != 'Fighting')]),'penalties (not fighting)')
print('There were',len(testDF.loc[testDF['Event'] == 'Goal']),'goals')
print('There were',len(testDF.loc[(testDF['Event'] == 'Goal') & ((testDF['Home Team Skaters'] == 6) | (testDF['Away Team Skaters'] == 6))]),'goals with 6 men')
print('There were',len(testDF.loc[(testDF['Event'] == 'Goal') & ((testDF['Home Team Skaters'] < 5) | (testDF['Away Team Skaters'] < 5))]),'goals with on PPs')


#testDF.loc[(testDF['Event'] == 'Takeaway') & (testDF['X Coordinate'] < 75)]

There were 387 penalties (not fighting)
There were 293 goals
There were 16 goals with 6 men
There were 74 goals with on PPs


In [439]:
testDF.loc[testDF['Event'] == 'Penalty Taken','Detail 1'].unique()

array(['Cross-checking', 'Holding', 'Fighting', 'High-sticking',
       'Tripping', 'Instigator', 'Hooking', 'Too many men on the ice',
       'Roughing', 'Interference', 'Slashing', 'Delay of game',
       'Holding the stick', 'Illegal Check to the Head', 'Diving',
       'Unsportsmanlike conduct', 'Illegal equipment',
       'Checking from behind', 'Goalkeeper interference', 'Misconduct',
       'Kneeing', 'Boarding', 'Spearing', 'Elbowing'], dtype=object)

In [456]:
print('There were',testDF.loc[(testDF['onPP']) | (testDF['onPK']),'TimeBetweenEvents'].sum(),'penalty seconds')
print('So there were',74/387, 'goals/power play')
print('Teams combine to score at a rate of',203*120/133261,'per 2 minutes')

There were 10739.0 penalty seconds
So there were 0.19121447028423771 goals/power play
Teams combine to score at a rate of 0.1827991685489378 per 2 minutes


In [451]:
40*20*3*60 - 10739

133261